In [1]:
# Improvement to the alignment and masking function

In [2]:
import intake
from cmip6_omz.utils import cmip6_collection
from cmip6_preprocessing.preprocessing import combined_preprocessing

In [3]:
# if this does not work on jupyter.rc, we can add some logic to 
col = intake.open_esm_datastore(cmip6_collection(zarr=True)) #TODO: Check with nc files

z_kwargs={"decode_times": True, "use_cftime": True, "consolidated": True}
n_kwargs={"decode_times": True, "use_cftime": True, 'chunks':{'time':3}}

variable_ids = ["thetao", "so", "o2", "agessc"] #"mlotst"
metric_variable_ids = ["thkcello", "areacello"] #"mlotst"

# models = o2_models()
# models = ['CanESM5-CanOE', 'GFDL-ESM4','CanESM5']# # shorter test run....'IPSL-CM6A-LR',
# models = [m for m in o2_models() if 'GFDL-ESM4' in m or 'Nor' in m]
models = ['GFDL-ESM4']

cat = col.search(
    source_id = models,
    grid_label=["gn"],
    experiment_id=["historical"],
    table_id=["Omon"],
    variable_id=['thetao', 'so'],
)
ds_dict = cat.to_dataset_dict(
        aggregate=False,
        zarr_kwargs=z_kwargs,
        cdf_kwargs=n_kwargs,
        preprocess=combined_preprocessing,
    )


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version.zstore'


In [4]:
list(ds_dict.keys())

['CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon.thetao.gn.v20190726./tigress/GEOCLIM/LRGROUP/jbusecke/projects/cmip_data_management_princeton/builder/../zarr_conversion/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gn/v20190726/CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon.thetao.gn.v20190726.zarr',
 'CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon.so.gn.v20190726./tigress/GEOCLIM/LRGROUP/jbusecke/projects/cmip_data_management_princeton/builder/../zarr_conversion/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/so/gn/v20190726/CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon.so.gn.v20190726.zarr']

In [5]:
from cmip6_preprocessing.postprocessing import merge_variables

In [6]:
dd = merge_variables(ds_dict)

In [7]:
dd

{'GFDL-ESM4.gn.historical.Omon.r1i1p1f1': <xarray.Dataset>
 Dimensions:        (x: 720, y: 576, vertex: 4, bnds: 2, lev: 35, time: 1980)
 Coordinates: (12/14)
   * x              (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
   * y              (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
     lat            (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
     lat_verticies  (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
   * bnds           (bnds) float64 1.0 2.0
   * lev            (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
     ...             ...
     lon_verticies  (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
   * time           (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
     time_bounds    (time, bnds) object dask.array<chunksize=(3, 2), meta=np.ndarray>
   * vertex         (vertex) int64 0 1 2 3
     lon_bounds     (bnds, y, x) float32 dask.

In [8]:
# can I wrap all these stupid masking functions into one?

from xarrayutils.utils import remove_bottom_values

ds = dd['GFDL-ESM4.gn.historical.Omon.r1i1p1f1'].unify_chunks()
ds

<xarray.Dataset>
Dimensions:        (x: 720, y: 576, vertex: 4, bnds: 2, lev: 35, time: 1980)
Coordinates: (12/14)
  * x              (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * y              (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
    lat            (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lat_verticies  (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
  * bnds           (bnds) float64 1.0 2.0
  * lev            (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    ...             ...
    lon_verticies  (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
  * time           (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bounds    (time, bnds) object dask.array<chunksize=(3, 1), meta=np.ndarray>
  * vertex         (vertex) int64 0 1 2 3
    lon_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>
    lat_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>
Data variables:
    thetao         (time, lev, y, x) float32 dask.array<chunksize=(3, 35, 576, 720), meta=np.ndarray>
    so             (time, lev, y, x) float32 dask.array<chunksize=(3, 35, 576, 720), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   [0.0]
    branch_time_in_parent:  [36500.0]
    comment:                <null ref>
    ...                     ...
    table_id:               Omon
    title:                  NOAA GFDL GFDL-ESM4 model output prepared for CMI...
    variant_info:           N/A
    variant_label:          r1i1p1f1
    intake_esm_varname:     None
    original_key:           CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon...

In [9]:
test = remove_bottom_values(ds)

In [10]:
import numpy as np
data = np.array([0,4,5, np.nan])
# idx = np.max(np.where(~np.isnan(data)))
# data[idx] = np.nan

In [11]:
np.argwhere(~np.isnan(data))[-1][0]

2

In [74]:
# just code that shit in numba
from numba import float64, guvectorize
import numpy as np
import xarray as xr

@guvectorize(
    [
        (float64[:], float64[:]),
    ],
    "(n)->(n)",
    nopython=True,
)
def _remove_last_value(data, output):
    # initialize output
    output[:] = data[:]
    for i in range(len(data)-1):
        if np.isnan(output[i+1]):
            output[i] = np.nan
    # take care of boundaries
    if not np.isnan(output[-1]):
        output[-1] = np.nan

def remove_bottom_values_numba(da, dim='lev'):
    
    out = xr.apply_ufunc(
        _remove_last_value,
        da,
        input_core_dims=[[dim]],
        output_core_dims=[[dim]],
        dask="parallelized",
        output_dtypes=[da.dtype],
    )
    return out

def remove_bottom_values_recoded(ds, dim="lev", fill_val=-1e10):
    """Remove the deepest values that are not nan along the dimension `dim`"""
    # for now assume that values of `dim` increase along the dimension
    if ds[dim][0] > ds[dim][-1]:
        raise ValueError(
            f"It seems like `{dim}` has decreasing values. This is not supported yet. Please sort before."
        )
    else:
        ds_masked = xr.Dataset({va:remove_bottom_values_numba(ds[va]) for va in ds.data_vars})
        ds_masked = ds_masked.transpose(*tuple([di for di in ds.dims if di in ds_masked]))
        ds_masked = ds_masked.assign_coords({co:ds[co].transpose(*[di for di in ds.dims if di in ds[co]]) for co in ds.coords})
        ds_masked.attrs = ds.attrs
        ds_masked = ds_masked
        return ds_masked

In [75]:
test = remove_bottom_values(ds.isel(time=0)).load()
test_numba = remove_bottom_values_recoded(ds.isel(time=0)).load()

/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/dask/array/core.py:383: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/dask/array/core.py:383: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)
/scratch/gpfs2/jbusecke/conda_tigressdata/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future wi

In [76]:
xr.testing.assert_allclose(test, test_numba.transpose('lev', 'bnds', 'y', 'x', 'vertex'))

# this is hella dumb! Xarray should be able to "broadcast like"

In [24]:
remove_bottom_values(ds)

<xarray.Dataset>
Dimensions:        (time: 1980, lev: 35, y: 576, x: 720, vertex: 4, bnds: 2)
Coordinates: (12/14)
  * x              (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * y              (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
    lat            (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lat_verticies  (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
  * bnds           (bnds) float64 1.0 2.0
  * lev            (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    ...             ...
    lon_verticies  (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
  * time           (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bounds    (time, bnds) object dask.array<chunksize=(3, 1), meta=np.ndarray>
  * vertex         (vertex) int64 0 1 2 3
    lon_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>
    lat_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>
Data variables:
    thetao         (time, lev, y, x) float32 dask.array<chunksize=(3, 35, 576, 720), meta=np.ndarray>
    so             (time, lev, y, x) float32 dask.array<chunksize=(3, 35, 576, 720), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   [0.0]
    branch_time_in_parent:  [36500.0]
    comment:                <null ref>
    ...                     ...
    table_id:               Omon
    title:                  NOAA GFDL GFDL-ESM4 model output prepared for CMI...
    variant_info:           N/A
    variant_label:          r1i1p1f1
    intake_esm_varname:     None
    original_key:           CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon...

In [22]:
test_single = remove_bottom_values_recoded(ds).so.isel(x=200, y=300, time=0).load()
test_single

<xarray.DataArray 'so' (lev: 35)>
array([34.2798233 , 34.27929306, 34.27620697, 34.27174759, 34.27720642,
       34.72245789, 35.08293915, 35.00019073, 34.66518021, 34.30405426,
       34.2118721 , 34.22891235, 34.28136063, 34.30194473, 34.30952835,
       34.31917191, 34.34711456, 34.39120102, 34.43114471, 34.46646881,
       34.49689865, 34.51949692, 34.53760529, 34.55934143, 34.58655167,
       34.61545181, 34.64251328, 34.65794754, 34.66550827,         nan,
               nan,         nan,         nan,         nan,         nan])
Coordinates:
    x        float64 -199.8
    y        float64 10.39
    lat      float32 10.39
  * lev      (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
    lon      float32 160.2
    time     object 1850-01-16 12:00:00

In [19]:
ds.so.isel(x=200, y=300, time=0).load()

<xarray.DataArray 'so' (lev: 35)>
array([34.279823, 34.279293, 34.276207, 34.271748, 34.277206, 34.722458,
       35.08294 , 35.00019 , 34.66518 , 34.304054, 34.211872, 34.228912,
       34.28136 , 34.301945, 34.30953 , 34.31917 , 34.347115, 34.3912  ,
       34.431145, 34.46647 , 34.4969  , 34.519497, 34.537605, 34.55934 ,
       34.58655 , 34.61545 , 34.642513, 34.657948, 34.66551 , 34.66648 ,
             nan,       nan,       nan,       nan,       nan],
      dtype=float32)
Coordinates:
    x        float64 -199.8
    y        float64 10.39
    lat      float32 10.39
  * lev      (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
    lon      float32 160.2
    time     object 1850-01-16 12:00:00
Attributes:
    cell_measures:  area: areacello volume: volcello
    cell_methods:   area: mean where sea time: mean
    long_name:      Sea Water Salinity
    original_name:  so
    standard_name:  sea_water_salinity
    units:          0.001

In [33]:
test.so.isel(x=200, y=300, time=0).load()

<xarray.DataArray 'so' (lev: 35)>
array([34.279823, 34.279293, 34.276207, 34.271748, 34.277206, 34.722458,
       35.08294 , 35.00019 , 34.66518 , 34.304054, 34.211872, 34.228912,
       34.28136 , 34.301945, 34.30953 , 34.31917 , 34.347115, 34.3912  ,
       34.431145, 34.46647 , 34.4969  , 34.519497, 34.537605, 34.55934 ,
       34.58655 , 34.61545 , 34.642513, 34.657948, 34.66551 ,       nan,
             nan,       nan,       nan,       nan,       nan],
      dtype=float32)
Coordinates:
    x        float64 -199.8
    y        float64 10.39
    lat      float32 10.39
  * lev      (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
    lon      float32 160.2
    time     object 1850-01-16 12:00:00
Attributes:
    cell_measures:  area: areacello volume: volcello
    cell_methods:   area: mean where sea time: mean
    long_name:      Sea Water Salinity
    original_name:  so
    standard_name:  sea_water_salinity
    units:          0.001

In [28]:
def mult(ds):
    return ds*100

In [30]:
ds.map_blocks(mult)

<xarray.Dataset>
Dimensions:        (y: 576, lev: 35, x: 720, vertex: 4, time: 1980, bnds: 2)
Coordinates: (12/14)
  * y              (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * lev            (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
  * x              (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * vertex         (vertex) int64 0 1 2 3
  * time           (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * bnds           (bnds) float64 1.0 2.0
    ...             ...
    lev_bounds     (lev, bnds) float64 dask.array<chunksize=(35, 1), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lon_verticies  (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
    time_bounds    (time, bnds) object dask.array<chunksize=(3, 1), meta=np.ndarray>
    lon_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>
    lat_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>
Data variables:
    so             (time, lev, y, x) float32 dask.array<chunksize=(3, 35, 576, 720), meta=np.ndarray>
    thetao         (time, lev, y, x) float32 dask.array<chunksize=(3, 35, 576, 720), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   [0.0]
    branch_time_in_parent:  [36500.0]
    comment:                <null ref>
    ...                     ...
    table_id:               Omon
    title:                  NOAA GFDL GFDL-ESM4 model output prepared for CMI...
    variant_info:           N/A
    variant_label:          r1i1p1f1
    intake_esm_varname:     None
    original_key:           CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Omon...